In [1]:
import pandas
import numpy as np
from sklearn import metrics
from sklearn import model_selection as cv
from sklearn import preprocessing


data = pandas.read_csv('employee_data_plus_gender.csv')
le = preprocessing.LabelEncoder()

data["Name"] = data["Name"].map(str.strip)
data["Title"] = data["Title"].map(str.strip)
data["Wage"] = data["Wage"].map(str.strip)
data['Wage'] = data['Wage'].replace(',', '',regex=True)

#0 is female, 1 is male
data.Gender = le.fit_transform(data.Gender)
data.Title = le.fit_transform(data.Title)

attribs = ["Wage", "Rate", "Title"]

(train, test) = cv.train_test_split(data, test_size = 0.2)



In [2]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(train[attribs],train["Gender"])
preds = rf.predict(test[attribs])
print(metrics.accuracy_score(test["Gender"], preds))

0.807692307692


In [4]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

clf = RandomForestClassifier()
# specify parameters and distributions to sample from
param_dist = {"max_depth": [2,3,5, None],
              "max_features": sp_randint(1, 3),
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}
# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,n_iter=n_iter_search)

random_search.fit(train[attribs], train["Gender"])
print(random_search.best_params_)
print(random_search.best_score_)
print(random_search.best_estimator_)

{'criterion': 'entropy', 'bootstrap': False, 'max_depth': 2, 'min_samples_split': 5, 'max_features': 2, 'min_samples_leaf': 8}
0.754901960784
RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=2, max_features=2,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=8, min_samples_split=5,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
